In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import random
import time


In [ ]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Webcam setup
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Get webcam dimensions
ret, frame = cap.read()
if not ret:
    print("Error: Could not read frame.")
    cap.release()
    exit()
h, w, _ = frame.shape

In [ ]:
# Define Dalgona shapes
def draw_shape(img, shape_type, color=(0, 255, 255), thickness=2, glow=False):
    center = (w//2, h//2)
    if shape_type == "circle":
        cv2.circle(img, center, 100, color, thickness)
        if glow and thickness != -1:
            cv2.circle(img, center, 102, (0, 255, 255, 50), 4)
    elif shape_type == "triangle":
        points = np.array([
            [center[0], center[1]-100],
            [center[0]-87, center[1]+50],
            [center[0]+87, center[1]+50]
        ], np.int32)
        if thickness == -1:
            cv2.fillPoly(img, [points], color)
        else:
            cv2.polylines(img, [points], True, color, thickness)
            if glow:
                cv2.polylines(img, [points], True, (0, 255, 255, 50), thickness+2)
    elif shape_type == "star":
        points = []
        for i in range(5):
            angle = np.pi/2 - i * 2 * np.pi / 5
            points.append([center[0] + 100 * np.cos(angle), center[1] - 100 * np.sin(angle)])
            angle += np.pi / 5
            points.append([center[0] + 50 * np.cos(angle), center[1] - 50 * np.sin(angle)])
        points = np.array(points, np.int32)
        if thickness == -1:
            cv2.fillPoly(img, [points], color)
        else:
            cv2.polylines(img, [points], True, color, thickness)
            if glow:
                cv2.polylines(img, [points], True, (0, 255, 255, 50), thickness+2)
    elif shape_type == "umbrella":
        if thickness == -1:
            cv2.ellipse(img, center, (80, 80), 0, 0, 180, color, -1)
        else:
            cv2.ellipse(img, center, (80, 80), 0, 0, 180, color, thickness)
            if glow:
                cv2.ellipse(img, center, (82, 82), 0, 0, 180, (0, 255, 255, 50), thickness+2)
        handle = np.array([
            [center[0], center[1]],
            [center[0], center[1]+100]
        ], np.int32)
        handle_thickness = max(1, thickness) if thickness != -1 else 1
        cv2.polylines(img, [handle], False, color, handle_thickness)
        if glow and thickness != -1:
            cv2.polylines(img, [handle], False, (0, 255, 255, 50), handle_thickness+2)
    return img

In [ ]:
# Create shape mask for collision detection
def create_shape_mask(shape_type):
    mask = np.zeros((h, w), dtype=np.uint8)
    draw_shape(mask, shape_type, color=255, thickness=10)  # Thicker outline
    return mask

In [ ]:
# Draw text with shadow
def draw_text_with_shadow(img, text, pos, font, scale, color, thickness, shadow_offset=(2, 2)):
    shadow_pos = (pos[0] + shadow_offset[0], pos[1] + shadow_offset[1])
    cv2.putText(img, text, shadow_pos, font, scale, (0, 0, 0), thickness+1)
    cv2.putText(img, text, pos, font, scale, color, thickness)

In [ ]:
# Draw progress bar
def draw_progress_bar(img, progress, pos, size=(200, 20), color=(0, 255, 0)):
    x, y = pos
    cv2.rectangle(img, (x, y), (x+size[0], y+size[1]), (100, 100, 100), -1)
    fill_width = int(size[0] * progress)
    cv2.rectangle(img, (x, y), (x+fill_width, y+size[1]), color, -1)
    cv2.rectangle(img, (x, y), (x+size[0], y+size[1]), (255, 255, 255), 1)

In [ ]:
# Game state
game_state = "start"
shapes = ["circle", "triangle", "star", "umbrella"]
shape_type = random.choice(shapes)
shape_mask = create_shape_mask(shape_type)
carving_mask = np.zeros((h, w), dtype=np.uint8)
start_time = 0
time_limit = 10 * 60  # 10 minutes
game_over = False
win = False
fade_alpha = 0
mistake_count = 0
max_mistakes = 30

In [1]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    if game_state == "start":
        # Start screen
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.7, frame, 0.3, 0)
        draw_text_with_shadow(frame, "Dalgona Candy Challenge", (w//4, h//4), 
                             cv2.FONT_HERSHEY_DUPLEX, 1.5, (0, 255, 255), 2)
        draw_text_with_shadow(frame, f"Shape: {shape_type.capitalize()}", (w//4, h//3), 
                             cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 1)
        draw_text_with_shadow(frame, "Pinch fingers to carve along the outline", (w//4, h//2), 
                             cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)
        draw_text_with_shadow(frame, "Avoid carving inside the shape!", (w//4, h//2+40), 
                             cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)
        draw_text_with_shadow(frame, "Press any key to start", (w//4, h//2+80), 
                             cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 255, 0), 1)
        draw_shape(frame, shape_type, color=(0, 255, 255), thickness=3, glow=True)
        
        cv2.imshow("Dalgona Candy Challenge", frame)
        key = cv2.waitKey(1)
        if key != -1:  # Any key pressed
            game_state = "playing"
            start_time = time.time()
            carving_mask = np.zeros((h, w), dtype=np.uint8)
            game_over = False
            win = False
            mistake_count = 0
        continue
    
    # Process hand landmarks
    results = hands.process(rgb_frame)
    index_x, index_y = None, None
    if results.multi_hand_landmarks and not game_over:
        for hand_landmarks in results.multi_hand_landmarks:
            index_tip = hand_landmarks.landmark[8]
            thumb_tip = hand_landmarks.landmark[4]
            index_x, index_y = int(index_tip.x * w), int(index_tip.y * h)
            thumb_x, thumb_y = int(thumb_tip.x * w), int(thumb_tip.y * h)
            distance = np.sqrt((index_x - thumb_x)**2 + (index_y - thumb_y)**2)
            
            if distance < 30:
                cv2.circle(carving_mask, (index_x, index_y), 5, 255, -1)
                if 0 <= index_x < w and 0 <= index_y < h:
                    if shape_mask[index_y, index_x] > 0:
                        # Carving on outline, continue
                        pass
                    else:
                        # Carving outside outline, check if inside shape
                        temp_mask = np.zeros((h, w), dtype=np.uint8)
                        draw_shape(temp_mask, shape_type, color=255, thickness=-1)
                        if temp_mask[index_y, index_x] > 0:
                            mistake_count += 1
                            if mistake_count >= max_mistakes:
                                game_over = True
                                win = False
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    # Create candy background
    candy_bg = np.zeros((h, w, 3), dtype=np.uint8)
    cv2.circle(candy_bg, (w//2, h//2), 150, (139, 69, 19), -1)
    frame = cv2.addWeighted(frame, 0.7, candy_bg, 0.3, 0)
    
    # Overlay Dalgona shape
    overlay = frame.copy()
    draw_shape(overlay, shape_type, color=(255, 255, 0), thickness=3, glow=True)
    
    # Show carved areas with gradient
    carved_display = cv2.bitwise_and(overlay, overlay, mask=cv2.bitwise_not(carving_mask))
    uncarved_display = cv2.bitwise_and(overlay, overlay, mask=carving_mask)
    frame = cv2.addWeighted(carved_display, 1, uncarved_display, 0.5, 0)
    
    # Draw cursor if pinching
    if index_x is not None and index_y is not None and not game_over:
        cv2.circle(frame, (index_x, index_y), 8, (0, 255, 0), 1)
    
    # Display time, progress, and mistakes
    elapsed_time = time.time() - start_time
    time_left = max(0, time_limit - elapsed_time)
    minutes, seconds = divmod(int(time_left), 60)
    time_color = (0, 255, 0) if time_left > 60 else (0, 165, 255) if time_left > 30 else (0, 0, 255)
    draw_text_with_shadow(frame, f"Time: {minutes:02d}:{seconds:02d}", (10, 30), 
                         cv2.FONT_HERSHEY_DUPLEX, 0.8, time_color, 2)
    draw_text_with_shadow(frame, f"Mistakes Left: {max_mistakes - mistake_count}", (10, 80), 
                         cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 0), 2)
    
    # Progress calculation
    dilated_carving = cv2.dilate(carving_mask, np.ones((5,5), np.uint8))
    overlap = cv2.bitwise_and(dilated_carving, shape_mask)
    progress = np.sum(overlap) / np.sum(shape_mask) if np.sum(shape_mask) > 0 else 0
    draw_progress_bar(frame, progress, (10, 50), color=(0, 255, 0))
    
    # Check win condition
    if not game_over and progress > 0.95:
        game_over = True
        win = True
    
    # Check time limit
    if time_left <= 0 and not game_over:
        game_over = True
        win = False
    
    # Game over screen with fade effect
    if game_over:
        fade_alpha = min(fade_alpha + 0.05, 0.7)
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, fade_alpha, frame, 1-fade_alpha, 0)
        text = "You Win!" if win else "Candy Broken!" if not win else "Time's Up!"
        text_color = (0, 255, 0) if win else (0, 0, 255)
        draw_text_with_shadow(frame, text, (w//4, h//2), cv2.FONT_HERSHEY_DUPLEX, 2, text_color, 3)
        draw_text_with_shadow(frame, "Press 'r' to restart or 'q' to quit", 
                             (w//4, h//2+50), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)
    
    # Show frame
    cv2.imshow("Dalgona Candy Challenge", frame)
    
    # Handle input
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r') and game_over:
        shape_type = random.choice(shapes)
        shape_mask = create_shape_mask(shape_type)
        carving_mask = np.zeros((h, w), dtype=np.uint8)
        start_time = time.time()
        game_over = False
        win = False
        fade_alpha = 0
        game_state = "start"
        mistake_count = 0



In [ ]:
# Cleanup
cap.release()
cv2.destroyAllWindows()
hands.close()